# Understanding the Core Objects in Deeplay

Before starting to implement applications in Deeplay, let's define the most important objects for building a model in Deeplay:
- **Application:** The main object that defines the neural network architecture and training process. This includes the loss, the optimizer, and the training logic. They are typically task-oriented, such as `ImageClassifier`, `ObjectDetector`.
- **Model:** A model is a specific neural network architecture that is part of an application. Examples include `ResNet`, `VGG`.
- **Component:** A model is usually made by combining multiple components. Components are much more flexible than models. Examples include `MultiLayerPerceptron`, `ConvolutionalEncoder2d`.
- **Block:** A block is a specific combination of layers that performs a small unit of calculation (for example `layer->activation`). Blocks are the building blocks of components, and are the most flexible objects in Deeplay. Examples include `LinearBlock`, `Conv2dBlock`.
- **Layer:** A layer consists of a single torch layer, such as `torch.nn.Linear`, `torch.nn.Conv2d`. Layers are the most basic building blocks in Deeplay.

In the following sections, you'll create some examples of these obejcts.

## Importing Deeplay

Import `deeplay` (shortened to `dl`, as an abbreaviation of both `deeplay` and `deeplearning`) ... 

In [1]:
import deeplay as dl

... and the `torch.nn` namespace.

In [2]:
import torch.nn as nn

## Creating Layers

Starting with the most basic building block, a `Layer` is a single PyTorch layer. In this example, you'll create a linear layer with 10 input features and 5 output features.

In [3]:
linear_layer = dl.Layer(nn.Linear, in_features=10, out_features=5)

print(linear_layer)

Layer[Linear](in_features=10, out_features=5)


You can modify the layer after it's created using the `configure()` method to change any of its properties. For example, here you'll change the number of output features and remove the bias.

In [4]:
linear_layer.configure(out_features=3, bias=False)

print(linear_layer)

Layer[Linear](in_features=10, out_features=3, bias=False)


To make the layer into a pure PyTorch module, you simply need to build it.

**NOTE:** Most Deeplay objects are modified in place when you build them. If you want to keep the original object, either call `.create()` or `.new().build()` instead; in this way, layers will return torch objects, while most other deeplay objects will return Deeplay objects.

In [5]:
torch_layer = linear_layer.build()

print(torch_layer)

Linear(in_features=10, out_features=3, bias=False)


## Creating Blocks

Going up one level in the hierarchy, a `Block` is a combination of layers that performs a relatively simple calculation. They are usually a sequence of `Layer` objects, but can include other blocks or sequences of blocks as well. In this example, you'll create a block that consists of a linear layer followed by a ReLU activation function.

In [6]:
linear_block = dl.LinearBlock(in_features=10, out_features=5)

print(linear_block)

AttributeError: module 'deeplay' has no attribute 'LinearBlock'

There is no activation by default. There are a few ways to add an activation to a block. The most common way is to use the `activation` argument when creating the block ...

In [ ]:
linear_block.activated(nn.ReLU)

print(linear_block)

... you can also add an activation to an existing block using the `activated()` method ...

In [ ]:
linear_block_with_activation = dl.LinearBlock(in_features=10, out_features=5, activation=dl.Layer(nn.ReLU)).build()

print(linear_block_with_activation)

... finally you can use `.configure(activation=...)` to add an activation to a block (this final way is rarely needed).

## Creating Components

A `Component` is a collection of blocks that are combined to form a more complex neural network component. In this example, you'll create a simple feedforward neural network component with two linear blocks, each followed by a ReLU activation function.

In [7]:
mlp_component = dl.MultiLayerPerceptron(
    in_features=10,
    hidden_features=[32],
    out_features=5,
    out_activation=dl.Layer(nn.ReLU)
)   

print(mlp_component)

MultiLayerPerceptron(
  (blocks): LayerList(
    (0): LayerActivationNormalizationDropout(
      (layer): Layer[Linear](in_features=10, out_features=32)
      (activation): Layer[ReLU]()
      (normalization): Layer[Identity](num_features=32)
      (dropout): Layer[Dropout](p=0)
    )
    (1): LayerActivationNormalizationDropout(
      (layer): Layer[Linear](in_features=32, out_features=5)
      (activation): Layer[ReLU]()
      (normalization): Layer[Identity](num_features=5)
      (dropout): Layer[Identity]()
    )
  )
)


Since the component is made of blocks, you can access the blocks using the `blocks` attribute. This allows you to modify the blocks after the component has been created. For example, you can change the activation function of the first block to a Sigmoid function ...

In [8]:
mlp_component.blocks[0].activated(nn.Sigmoid)

print(mlp_component)

AttributeError: 'LayerActivationNormalizationDropout' object has no attribute 'activated'

... or you can remove the activation.

In [9]:
mlp_component.blocks[0].remove("activation")

print(mlp_component)

MultiLayerPerceptron(
  (blocks): LayerList(
    (0): LayerActivationNormalizationDropout(
      (layer): Layer[Linear](in_features=10, out_features=32)
      (normalization): Layer[Identity](num_features=32)
      (dropout): Layer[Dropout](p=0)
    )
    (1): LayerActivationNormalizationDropout(
      (layer): Layer[Linear](in_features=32, out_features=5)
      (activation): Layer[ReLU]()
      (normalization): Layer[Identity](num_features=5)
      (dropout): Layer[Identity]()
    )
  )
)


## Creating Models

The next step up in the hierarchy is a `Model`, which is a specific neural network architecture. Examples would be lecun-5, resnet. For fully connected networks, there are few widely recognized standard models. Here, you'll simply instantiate a small MLP model.

In [10]:
small_mlp = dl.models.SmallMLP(in_features=10, out_features=5)

print(small_mlp)

AttributeError: module 'deeplay.models' has no attribute 'SmallMLP'

## Creating Applications

Finally, an `Application` is the highest level of abstraction in Deeplay. It defines the neural network architecture and training process, including the loss, optimizer, and training logic. In this example, you'll create a simple classifier application that uses the previously defined model.

In [11]:
classifier = dl.Classifier(small_mlp, optimizer=dl.Adam(lr=0.001))

NameError: name 'small_mlp' is not defined

The Deeplay optimizer is a wrapper around the torch optimizer which delays the attribution of parameters from the models until after the model is actually created.